In [3]:
import numpy as np
import pandas as pd

iris = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
labels, species = pd.factorize(iris.species)
X = iris.copy()
print(X.shape)
X.species = labels
X.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
X_train = X.sample(frac=0.8)
test = X.drop(X_train.index)
X_test = test.drop(columns="species")
y_test = test["species"]

In [7]:
y_test.shape

(30,)

In [8]:
class Node:
    
    def __init__(self):
        self.predict_ = None
        self.feature_ = None
        self.sep_ = None
        self.left_ = None
        self.right_ = None
    
    def save(self, feature, sep):
        self.feature_ = feature
        self.sep_ = sep

In [9]:
class DecisionTree:
    
    def __init__(self):
        self.root_ = Node()
        
    def fit(self, df):
        self.df_ = df
        self.fit_r(self.root_, np.arange(self.df_.shape[0]))
        del self.df_
    
    def fit_r(self, nd, indices):
        classes = set(self.df_.iloc[indices, -1])
        if len(classes) == 1:
            nd.predict_ = list(classes)[0]
            return
        li, ri, j, s, ig = [None, None, None, None, 0]
        for feature in self.df_.columns[:-1]:
            for sep in np.unique(self.df_[feature]):
                left, right = self.split(indices, feature, sep)
                curr_ig = self.IG(indices, left, right, feature, sep)
                if curr_ig > ig:
                    li, ri, j, s, ig = left, right, feature, sep, curr_ig
        nd.save(j, s)
        nd.left_ = Node()
        if li.shape[0] > 5:
            self.fit_r(nd.left_, li)
        else:
            nd.left_.predict_ = np.argmax(np.bincount(self.df_.iloc[li, -1]))
        
        nd.right_ = Node()
        if ri.shape[0] > 5:
            self.fit_r(nd.right_, ri)
        else:
            nd.right_.predict_ = np.argmax(np.bincount(self.df_.iloc[ri, -1]))
            
    
    def split(self, indices, feature, sep):
        tf_array = (self.df_.iloc[indices,list(self.df_.columns).index(feature)] > sep).values
        return indices[(1-tf_array).astype(bool)], indices[tf_array]
    
    def gini_index(self, indices):
        labels = self.df_.iloc[indices,-1].values
        p = np.bincount(labels) / len(labels)
        return 1 - p @ p # (p**2).sum()
    
    def IG(self, fi, li, ri, feature, sep):
        return self.gini_index(fi) - (li.shape[0] * self.gini_index(li) + ri.shape[0] * self.gini_index(ri)) / len(fi)
    
    def predict(self, df):
        return np.array([self.pr(row[1], self.root_) for row in df.iterrows()])
    
    def pr(self, array, node):
        if node.predict_ is None:
            return self.pr(array, node.right_ if array[node.feature_] > node.sep_ else node.left_)
        else:
            return node.predict_

In [10]:
dt = DecisionTree()
dt.fit(X_train)

In [11]:
print(dt.predict(X_test))

[0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 2 1 1 1 2 2 2 2 2 2 1 2 2 2]


In [12]:
print(np.array(y_test))

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]


In [13]:
dt.predict(X_test) - np.array(y_test)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0], dtype=int64)